In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Kannada-MNIST/sample_submission.csv
/kaggle/input/Kannada-MNIST/Dig-MNIST.csv
/kaggle/input/Kannada-MNIST/train.csv
/kaggle/input/Kannada-MNIST/test.csv


In [15]:
import tensorflow as tf
import tensorflow.keras as keras

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input, BatchNormalization, Dropout, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [18]:
train = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
test = pd.read_csv('/kaggle/input/Kannada-MNIST/Dig-MNIST.csv')
Y_train = train['label']
X_train = train.drop(['label'],axis=1)
Y_test = test['label']
X_test = test.drop(['label'],axis=1)
X_train = X_train.values.reshape(-1,28, 28,1)
X_test = X_test.values.reshape(-1,28, 28,1)
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.2)
# Y_train = to_categorical(Y_train, num_classes = 10)
# Y_val = to_categorical(Y_val, num_classes = 10)
# Y_test = to_categorical(Y_test, num_classes = 10)

In [6]:
num_classes = 10
input_shape = (28, 28, 1)

In [17]:
X_train.shape

(48000, 28, 28, 1)

In [19]:
model = Sequential(
    [
        Input(shape=input_shape),
        Conv2D(32, kernel_size=(3, 3), activation="relu"),
        MaxPool2D((2,2)),
        Conv2D(64, kernel_size=(3, 3), activation="relu"),
        MaxPool2D((2,2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(num_classes, activation="softmax")
    ]
)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               409856    
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [21]:
batch_size = 100
epochs = 15

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
432/432 [==============================] - 2s 4ms/step - loss: 0.5828 - accuracy: 0.9504 - val_loss: 0.0819 - val_accuracy: 0.9781
Epoch 2/15
432/432 [==============================] - 2s 4ms/step - loss: 0.0352 - accuracy: 0.9889 - val_loss: 0.0784 - val_accuracy: 0.9779
Epoch 3/15
432/432 [==============================] - 2s 4ms/step - loss: 0.0217 - accuracy: 0.9928 - val_loss: 0.0556 - val_accuracy: 0.9856
Epoch 4/15
432/432 [==============================] - 1s 3ms/step - loss: 0.0157 - accuracy: 0.9945 - val_loss: 0.0848 - val_accuracy: 0.9846
Epoch 5/15
432/432 [==============================] - 2s 3ms/step - loss: 0.0140 - accuracy: 0.9957 - val_loss: 0.0573 - val_accuracy: 0.9894
Epoch 6/15
432/432 [==============================] - 1s 3ms/step - loss: 0.0127 - accuracy: 0.9957 - val_loss: 0.0635 - val_accuracy: 0.9869
Epoch 7/15
432/432 [==============================] - 1s 3ms/step - loss: 0.0204 - accuracy: 0.9940 - val_loss: 0.0570 - val_accuracy: 0.9885
Epoch 

In [22]:
model.evaluate(X_test, Y_test)

375/375 [==============================] - 1s 3ms/step - loss: 0.0464 - accuracy: 0.9911


[0.04639856517314911, 0.9910833239555359]

In [23]:
test1 = pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv')
test2 = test1.drop(['id'], axis=1)
test2 = test2.values.reshape(-1,28,28,1)
y_hat = model.predict(test2).argmax(axis=-1)
submission = pd.read_csv('/kaggle/input/Kannada-MNIST/sample_submission.csv')
submission['label']=y_hat
submission.to_csv('submission.csv', index=False)
submission.head()

,id,label
0,0,3
1,1,0
2,2,2
3,3,6
4,4,7
